In [ ]:
# Next step: try collecting data from 1/1/2018-3/31/2025 !

In [ ]:
# Try from this url:
url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-01&endDate=2025-03-31&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json'
# If that doesn't work, work on looping through all the dates and requesting batches!

In [ ]:
# note to self: update README with the url and output path when I have those and start wrapping this one up!

In [ ]:
# get a list of all the dates... and urls...

In [23]:
import datetime

In [35]:
startdate = datetime.date(2018,1,1)
enddate = datetime.date(2025,3,31)

# if I go this route, add a sleep timer to avoid bombarding their servers too much!
# and maybe modify this code to request in batches of like 1 month or 10 days at a time, rather than just 1 at a time?
# depends on how it goes trying to request the full 5-year-3-month chunk...

i = 1

requestdate = startdate
while requestdate <= enddate :
    url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate='
    url +=  str(requestdate)
    url += '&endDate='
    url += str(requestdate)
    url += '&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json'
    if i <= 5 :
        print(url)
    requestdate += datetime.timedelta(days=1)
    i += 1

print('... and so on...')
print('final url: ' + url)

https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-01&endDate=2020-01-01&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-02&endDate=2020-01-02&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-03&endDate=2020-01-03&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-04&endDate=2020-01-04&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2020-01-05&endDate=2020-01-05&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json
... and so on...
final url: https://www.ncei.noaa.gov/access/serv

In [ ]:
today = datetime.date()
tomorrow = today + datetime.timedelta(days=1)
print(tomorrow)

In [21]:
import datetime

today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
print(tomorrow)

2025-03-31


In [3]:
import requests
import matplotlib.pyplot as plt

In [4]:
with open('../data/apikeys.json') as fi:
    credentials = json.load(fi)

In [5]:
api_key = credentials['noaa']

In [6]:
endpoint = 'https://www.ncei.noaa.gov/access/services/search/v1/data?api_key=jrWndEKCwWnaUiPOzlOLDlVHDsnlCIXC&dataset=daily-summaries&startDate=2010-01-01T00:00:00&endDate=2016-12-31T23:09:59&boundingBox=35.462327,-82.563951,35.412327,-82.513951&dataTypes=PRCP&stations=USW00003812&limit=10&offset=0'

response = requests.get(endpoint)

In [11]:
response

<Response [400]>

In [13]:
import requests
import numpy as np
import pandas as pd
import json
from datetime import datetime
import matplotlib.pyplot as plt


Token = api_key

station_id = 'GHCND:USW00023129'
station_id1 = 'GHCND:USC00107689'
station_id2 = 'GHCND:USW00003122'

#initialize lists to store data
dates_temp = []
dates_prcp = []
temps = []

year='2024'
    
#this works!
#r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid='
#                +station_id1+'&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})

#try this: https://www.ncei.noaa.gov/cdo-web/api/v2/data?dataset=global-summary-of-the-day&stations=USW00014037&startDate=2025-03-28&endDate=2025-03-28&dataTypes=TMAX,TMIN,PRCP&format=json
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?dataset=global-summary-of-the-day&dataTypes=TMAX,TMIN,PRCP&stationid='
                +station_id1+'&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    
    #load the api response as a json
r

<Response [400]>

In [17]:
token = api_key
headers = {"token":token}

# works!
# url = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-summary-of-the-year&dataTypes=DP01,DP05,DP10,DSND,DSNW,DT00,DT32,DX32,DX70,DX90,SNOW,PRCP&stations=ASN00084027&startDate=1952-01-01&endDate=1970-12-31&includeAttributes=true&format=json"

# this works!  But gives TMAX of 100, TMIN of 44, and PRCP of 38?!?!?
#url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USC00406403&startDate=2024-03-02&endDate=2024-03-02&dataTypes=TMAX,TMIN,PRCP&format=json'

#!!!!
# this works!  TMAX 167, TMIN 128, PRCP 117 (10ths of degrees celcius, and 10th of mm of rainfall)
#url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2024-03-06&endDate=2024-03-06&dataTypes=TMAX,TMIN,PRCP&format=json'
url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2024-03-05&endDate=2024-03-06&dataTypes=TMAX,TMIN,PRCP&units=standard&format=json'
#url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00013897&startDate=2024-03-06&endDate=2024-03-06&format=json'


# url = 'https://www.ncei.noaa.gov/access/services/data/v1'

r = requests.get(url, headers = headers)
print(r)
print(r.text)


<Response [200]>
[
{"DATE":"2024-03-05","STATION":"USW00013897","TMAX":"64","TMIN":"58","PRCP":"0.52"}
,{"DATE":"2024-03-06","STATION":"USW00013897","TMAX":"62","TMIN":"55","PRCP":"0.46"}
]



In [21]:

url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/datasets'

r = requests.get(url, headers = headers)
print(r)
print(r.text)

<Response [200]>
{"metadata":{"resultset":{"offset":1,"count":11,"limit":25}},"results":[{"uid":"gov.noaa.ncdc:C00861","mindate":"1763-01-01","maxdate":"2025-03-28","name":"Daily Summaries","datacoverage":1,"id":"GHCND"},{"uid":"gov.noaa.ncdc:C00946","mindate":"1763-01-01","maxdate":"2025-03-01","name":"Global Summary of the Month","datacoverage":1,"id":"GSOM"},{"uid":"gov.noaa.ncdc:C00947","mindate":"1763-01-01","maxdate":"2025-01-01","name":"Global Summary of the Year","datacoverage":1,"id":"GSOY"},{"uid":"gov.noaa.ncdc:C00345","mindate":"1991-06-05","maxdate":"2025-03-29","name":"Weather Radar (Level II)","datacoverage":0.95,"id":"NEXRAD2"},{"uid":"gov.noaa.ncdc:C00708","mindate":"1994-05-20","maxdate":"2025-03-25","name":"Weather Radar (Level III)","datacoverage":0.95,"id":"NEXRAD3"},{"uid":"gov.noaa.ncdc:C00821","mindate":"2010-01-01","maxdate":"2010-01-01","name":"Normals Annual/Seasonal","datacoverage":1,"id":"NORMAL_ANN"},{"uid":"gov.noaa.ncdc:C00823","mindate":"2010-01-01","ma

In [23]:
# Daily Summaries GHCND
# Normals Hourly NORMAL_HLY
# Precipitation Hourly PRECIP_HLY

url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/stations?locationid=FIPS:47&limit=1000'

r = requests.get(url, headers = headers)
print(r)
#print(r.text)

<Response [200]>


In [25]:
token = api_key

url = "http://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCNDMS&startdate=2015-03-03&enddate=2025-03-03"
headers = {"token":token}

r = requests.get(url, "dataset", headers = headers).text
r

'{"metadata":{"resultset":{"offset":1,"count":3988478,"limit":25}},"results":[{"date":"2015-04-01T00:00:00","datatype":"CLDD","station":"GHCND:AE000041196","attributes":"12,6","value":565},{"date":"2015-04-01T00:00:00","datatype":"DP01","station":"GHCND:AE000041196","attributes":"0,0","value":1},{"date":"2015-04-01T00:00:00","datatype":"DP05","station":"GHCND:AE000041196","attributes":"0,0","value":1},{"date":"2015-04-01T00:00:00","datatype":"DP10","station":"GHCND:AE000041196","attributes":"0,0","value":1},{"date":"2015-04-01T00:00:00","datatype":"DT00","station":"GHCND:AE000041196","attributes":"16,6","value":0},{"date":"2015-04-01T00:00:00","datatype":"DT32","station":"GHCND:AE000041196","attributes":"16,6","value":0},{"date":"2015-04-01T00:00:00","datatype":"DT90","station":"GHCND:AE000041196","attributes":"14,7","value":9},{"date":"2015-04-01T00:00:00","datatype":"DX32","station":"GHCND:AE000041196","attributes":"14,7","value":0},{"date":"2015-04-01T00:00:00","datatype":"EMNT","st